# KNN desde cero

### _**"Los datos son el registro de todo lo que ocurre. El ML nos ayuda a descubrir patrones en los datos, que es donde vive el significado, el verdadero valor"**_

In [1]:
import pandas as pd
dc_listings = pd.read_csv("dc_airbnb.csv")
dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state
0,92%,91%,26,4,Entire home/apt,1.0,1.0,2.0,$160.00,$115.00,$100.00,1,1125,0,38.890046,-77.002808,Washington,20003,DC
1,90%,100%,1,6,Entire home/apt,3.0,3.0,3.0,$350.00,$100.00,NaN,2,30,65,38.880413,-76.990485,Washington,20003,DC
2,90%,100%,2,1,Private room,1.0,2.0,1.0,$50.00,NaN,NaN,2,1125,1,38.955291,-76.986006,Hyattsville,20782,MD
3,100%,NaN,1,2,Private room,1.0,1.0,1.0,$95.00,NaN,NaN,1,1125,0,38.872134,-77.019639,Washington,20024,DC
4,92%,67%,1,4,Entire home/apt,1.0,1.0,1.0,$50.00,$15.00,$450.00,7,1125,0,38.996382,-77.041541,Silver Spring,20910,MD


### Ahora haremos un analisis exploratorio (EDA) de los datos.

In [3]:
dc_listings.isnull().sum()

host_response_rate       434
host_acceptance_rate     614
host_listings_count        0
accommodates               0
room_type                  0
bedrooms                  21
bathrooms                 27
beds                      11
price                      0
cleaning_fee            1388
security_deposit        2297
minimum_nights             0
maximum_nights             0
number_of_reviews          0
latitude                   0
longitude                  0
city                       0
zipcode                    9
state                      0
dtype: int64

In [10]:
dc_listings.describe()

,host_listings_count,accommodates,bedrooms,bathrooms,beds,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude
count,3723.000000,3723.000000,3702.000000,3696.000000,3712.000000,3723.000000,3.723000e+03,3723.000000,3723.000000,3723.000000
mean,13.517325,3.195004,1.210157,1.256358,1.643319,2.250067,5.803069e+05,15.306742,38.913967,-77.023294
std,64.534408,2.012216,0.839851,0.585539,1.182117,3.622879,3.519552e+07,29.645586,0.021647,0.026951
min,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000e+00,0.000000,38.825061,-77.110525
25%,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.200000e+02,1.000000,38.901789,-77.039859
50%,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.125000e+03,4.000000,38.913375,-77.026410
75%,3.000000,4.000000,1.000000,1.000000,2.000000,3.000000,1.125000e+03,16.000000,38.926509,-77.002798
max,480.000000,16.000000,10.000000,8.000000,16.000000,180.000000,2.147484e+09,362.000000,38.996382,-76.913137


### Eligiendo el valor de K y la metrica de similitud

Para este ejemplo usaremos $k = 5$ y se usara como metrica de similitud la Distancia Euclidiana.<br>
La **Distancia Euclideana** mide una linea recta entre el punto de consulta y otro punto. Cuando se trata<br>
de predecir un valor continuo, como en este caso _el precio_, la principal metrica de similitud es la<br>
**Distancia Euclideana**.

La **Distancia Euclideana** esta definida por la siguiente formula: $$ d = \sqrt{(q_{1}-p_{1})^{2} + (q_{2}-p_{2})^{2} + ... + (q_{n}-p_{n})^{2}} $$ 
Donde $ q_{1} $ a $ q_{n} $ representan los valores de las caracteristicas de una observacion y $ p_{1} $ a $ p_{n} $ <br>
representan los valors de las caracteristicas de la otra observacion.

#### Para este caso solo se usara una caracteristica. ( _**Caso univariante**_ )

Por lo que la formula se simplifica a $ d = |q_{1}-p_{1}| $.

In [12]:
import numpy as np

our_accomodates = 3
first_living_space_value = dc_listings.iloc[0]['accommodates']
first_distance = np.abs( first_living_space_value - our_accomodates )
print(f"La distancia euclidiana con la primer observacion es: {first_distance}")

La distancia euclidiana con la primer observacion es: 1


## ¿Como sabemos si esto es alto o bajo? <br>
Partiendo de la ecuacion que describe a la Distancia Euclideana, podemos concluir que el valor mas bajo que podemos alcanzar <br>
es cero. Esto cuando la caracteristica es la misma para las dos observaciones, por lo que cuanto la distancia ($d$) se acerque mas <br>
a cero, mas similares son las obseraciones, en este caso, los espacios vitales.

### Calculando la distancia entre cada una de las observaciones y nuestra observacion.

In [24]:
dc_listings['distance'] = dc_listings['accommodates'].apply( lambda x_acc: np.abs(x_acc - our_accomodates) )
dc_listings['distance'].value_counts()

distance
1     2294
2      503
0      461
3      279
5       73
4       35
7       22
6       17
9       12
13       8
8        7
12       6
11       4
10       2
Name: count, dtype: int64

El metodo value_counts(), devuelve una serie que contiene la frecuencia de cada fila distinta en el DataFrame.

### Aleatoridad y Clasificacion

Para que los espacios vitales con los que comparemos el nuestro, sean diferentes en cada prediccion. <br>
Aplicaremos aleatoridad a los datos.

In [55]:
dc_listings = dc_listings.loc[ np.random.permutation(len(dc_listings)) ] 
# print(dc_listings.loc[[21,234,2,556,87]])
# print(dc_listings)
dc_listings = dc_listings.sort_values('distance')
dc_listings.iloc[0:10]['price']
# dc_listings.head()

3356     $64.00
175     $119.00
315     $259.00
2414    $150.00
930     $100.00
274     $120.00
631     $108.00
2646    $400.00
2328     $89.00
3420    $150.00
Name: price, dtype: object

The permutation() method returns a re-arranged array (and leaves the original array un-changed).

Esto quiere decir que generara un ndarray del tamaño de la longitud de dc_listigns y con ayuda del .loc, tomaremos los valores en el orden <br>
del ndarray.
